# Extracting and Comparing Year Transit Facility Rebuilt From Notes

## Loading in Data

In this exercise I will need the regex (re) package, pandas, and natural language toolkit. 

In [1]:
import re
import pandas as pd
import nltk

I am pulling in a published dataset that my team would like to work with. Each record holds data on a single transit agency facility, reported as of 2019.

In [2]:
response='https://www.transit.dot.gov/sites/fta.dot.gov/files/2020-10/2019%20Facility%20Inventory.xlsx'
response
df=pd.read_excel(response)

There is a column for Year Built, as well as when a condition assessment was performed, and any notes that the reporting transit agency felt were relevant to include. I only care about these particular records (along with the name of the agency) and I only care about records that have notes provided.

In [3]:
al=df[['Agency Name','Year Built','Condition Assessment Date','Notes']]
Notes_only=al[al.Notes.notnull()] #filtering out all non-null notes
Notes_only

,Agency Name,Year Built,Condition Assessment Date,Notes
0,King County Department of Metro Transit,2017,NaT,"Administrative offices, call & dispatch center..."
5,King County Department of Metro Transit,2011,2015-01-01,Operations Building
9,King County Department of Metro Transit,1980,2015-01-01,Hostler building
12,King County Department of Metro Transit,2014,2015-01-01,Maintenance emergency spill response storage
13,King County Department of Metro Transit,2014,2015-01-01,Maintenance tire chain storage
...,...,...,...,...
13305,County of Hawaii Mass Transit Agency,2018,2018-06-30,Facility includes a 110'x150' Maintenance Area...
13306,Humboldt County,2016,2017-02-10,"Senior center, Admin, Passenger parking, and v..."
13308,"Pahrump Senior Center, Inc.",1960,2018-09-30,"Pahrump Senior Center, parking, passenger load..."
13315,City of El Segundo,2008,2018-10-01,"This facility was built by El Segundo, the lan..."


## Using Natural Language Processing to Extract Year

Now I will use regular expression (regex) tokenization to find the right string. I want to find a four digit year, which can be in either the 1900s or the 2000s. The first command below will limit the Notes_Year

In [39]:
Notes_year=Notes_only[Notes_only['Notes'].str.contains('1[0-9]{3}','20[0-9]{2}')] #497 have rows that are probably years
Notes_year['Notes']

166      1212 Sharp is a administrative offices. Major ...
425      Original agreement between IT and WSDOT began ...
478                                 47.178324, -122.677895
942                   Permanent Admin Facility  built 1898
945                Tower - Unoccupied/Abandoned Built 1875
                               ...                        
13038    combination of admin space (2963 sf) and maint...
13143    Facility type is hybrid Administrative and Mai...
13144    Facility type is hybrid Administrative and Mai...
13145    Facility type is hybrid Administrative and Mai...
13296    Lassen Rural Bus Office and Maintenance Bay.  ...
Name: Notes, Length: 200, dtype: object

In [24]:
Notes_year.head(1)

,Agency Name,Year Built,Condition Assessment Date,Notes
166,Spokane Transit Authority,2015,2017-04-28,1212 Sharp is a administrative offices. Major ...


This does not not show the entire notes field (is truncated). I'd like to quickly verify that they all have years, to verify my code above worked. I can use the option context function to display the max column width using the following.

In [26]:
with pd.option_context('display.max_colwidth', -1): display(Notes_year) #https://stackoverflow.com/questions/25351968/how-to-display-full-non-truncated-dataframe-information-in-html-when-convertin

,Agency Name,Year Built,Condition Assessment Date,Notes
166,Spokane Transit Authority,2015,2017-04-28,1212 Sharp is a administrative offices. Major remodel in 2015.
425,Intercity Transit,1983,2020-02-14,"Original agreement between IT and WSDOT began 10/24, 1983. Site was expanded per cooperative agreement dated 10/13, 2008. Condition assessment (2019) performed by IT Facilities Manager."
478,County of Pierce,1983,2019-04-24,"47.178324, -122.677895"
942,Massachusetts Bay Transportation Authority,1900,2019-06-29,Permanent Admin Facility built 1898
945,Massachusetts Bay Transportation Authority,1900,2019-06-29,Tower - Unoccupied/Abandoned Built 1875
...,...,...,...,...
13038,"Developmental Services of Northwest Kansas, Inc.",2005,2018-06-30,combination of admin space (2963 sf) and maint (14353 sf)
13143,Town of Telluride,2013,2018-08-15,Facility type is hybrid Administrative and Maintenance; phase 3 was in 2013. Phase 2 was in 1999 . Phase 1 was 1987
13144,Town of Telluride,1987,2018-08-15,Facility type is hybrid Administrative and Maintenance; originally built in 1987
13145,Town of Telluride,1999,2018-08-15,Facility type is hybrid Administrative and Maintenance; Phase 2 was in 1999


## Performing the (Imperfect) Comparison

Scanning through some of the notes above, it looks like the reported values in Year Built are very accurate compared to the notes. At this point, it isn't clear whether it is valuable to actually extract the year from the notes column to perform a logical comparison.

In [116]:
pattern = '([0-9]{4})'

Notes_year['result'] = Notes_year.loc[:,'Notes'].str.extract(pattern)
Notes_year['result']=to_numeric(Notes_year['result'])
Notes_year

/Users/Joe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Joe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Agency Name,Year Built,Condition Assessment Date,Notes,result,flagged,ok,agree
166,Spokane Transit Authority,2015,2017-04-28,1212 Sharp is a administrative offices. Major ...,1212,False,False,False
425,Intercity Transit,1983,2020-02-14,Original agreement between IT and WSDOT began ...,1983,True,True,True
478,County of Pierce,1983,2019-04-24,"47.178324, -122.677895",1783,False,False,False
942,Massachusetts Bay Transportation Authority,1900,2019-06-29,Permanent Admin Facility built 1898,1898,False,False,False
945,Massachusetts Bay Transportation Authority,1900,2019-06-29,Tower - Unoccupied/Abandoned Built 1875,1875,False,False,False
...,...,...,...,...,...,...,...,...
13038,"Developmental Services of Northwest Kansas, Inc.",2005,2018-06-30,combination of admin space (2963 sf) and maint...,2963,False,False,False
13143,Town of Telluride,2013,2018-08-15,Facility type is hybrid Administrative and Mai...,2013,True,True,True
13144,Town of Telluride,1987,2018-08-15,Facility type is hybrid Administrative and Mai...,1987,True,True,True
13145,Town of Telluride,1999,2018-08-15,Facility type is hybrid Administrative and Mai...,1999,True,True,True


So I can use an equivalence operator to add a new column, 'agree', which is based on true/false.

In [115]:
Notes_year['agree']=(Notes_year['result']==Notes_year['Year Built'])
Notes_year.value_counts('agree')

/Users/Joe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


agree
False    161
True      39
dtype: int64

I can see from the above that there are only 39 results that agree. This seems like too many based on what I saw before. Upon further investigation I find out there must be a constraint in the system limiting the Year Built to 1900 and afterwards.

In [117]:
Notes_year[Notes_year['result']>1900].value_counts('agree')

agree
False    52
True     38
dtype: int64

## Conclusion

Now we see a much more balanced dataset. Let's check this set to make sure this is worthwhile.

In [128]:
Notes_year[Notes_year['result']>1900][Notes_year['result']<2021][Notes_year['agree']==False]

/Users/Joe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/Users/Joe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Agency Name,Year Built,Condition Assessment Date,Notes,result,flagged,ok,agree
1757,Pioneer Valley Transit Authority,1985,2018-06-30,\nThe University of Massachusetts owned the or...,1974,False,False,False
1787,Housatonic Area Regional Transit,1998,2019-07-30,Administrative Office and Maintenance Facility...,1986,False,False,False
1789,Cape Ann Transportation Authority,2001,2018-06-30,"built in 1980, CATA purchased 2001 and rehabbe...",1980,False,False,False
1915,Connecticut Department of Transportation,1986,NaT,Original Lot Built in 1977 - Town Owned,1977,False,False,False
1917,Connecticut Department of Transportation,1991,NaT,Original Lot built in 1967 (Town Owned),1967,False,False,False
1939,Connecticut Department of Transportation,1992,NaT,Original Lot built in 1978 (Town Owned),1978,False,False,False
1944,Connecticut Department of Transportation,1996,NaT,Built in 1985 - State Owned,1985,False,False,False
2035,Niagara Frontier Transportation Authority,2003,2017-01-01,"Opened as Auditorium station Oct 9, 1984",1984,False,False,False
2036,Niagara Frontier Transportation Authority,2003,2017-01-01,"Opened as Auditorium station Oct 9, 1984",1984,False,False,False
2042,Niagara Frontier Transportation Authority,2016,2017-01-01,"Opened October 9, 1984 Rebuilt 7/5-21/14",1984,False,False,False


There are many values in this set that seem worth pursuing. Some are false positives as there were multiple values in the notes and the second one was right, where my query only captured the first one. What I would like to do in the future is go back and select the highest first, since "Year Built" is also defined as "Year Remodeled as New". In other words, the notes may be describing the original year built, which would not be reported.